# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
print(geoapify_key)

8880bbcb6b7f468a848236a8e8cf4216


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,272.15,82,21,3.73,CL,1690434685
1,1,bethel,41.3712,-73.4140,297.32,78,0,3.60,US,1690434685
2,2,waitangi,-43.9535,-176.5597,280.83,54,91,7.32,NZ,1690434685
3,3,avarua,-21.2078,-159.7750,294.18,83,20,1.54,CK,1690434685
4,4,gero,-8.7039,121.1881,300.41,56,12,3.09,ID,1690434685


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Plotting the points with the size of the point representing humidity
colormap = 'coolwarm'  # You can choose a different colormap as per your preference

# Plotting the points with the size of the point representing humidity
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    size='Humidity',
    hover_cols=['City', 'Humidity'],
    geo=True,
    tiles='CartoLight',
    frame_width=1000,
    frame_height=650,
    color='Humidity',  # Set marker color based on humidity
    cmap=colormap  # Use the specified colormap
)

# Display the map
# YOUR CODE HERE

# Customize the plot (optional)
map_plot = map_plot.opts(title='City Humidity Map')

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
max_temp_range = (21, 27)  # Temperature range in Celsius
max_humidity = 100  # Maximum humidity percentage
max_cloudiness = 0  # Cloudiness should be 0 (completely clear sky)
max_wind_speed = 4.5  # Maximum wind speed in m/s

# Narrow down the DataFrame based on the criteria
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= max_temp_range[0]) &
    (city_data_df['Max Temp'] <= max_temp_range[1]) &
    (city_data_df['Humidity'] <= max_humidity) &
    (city_data_df['Cloudiness'] == max_cloudiness) &
    (city_data_df['Wind Speed'] <= max_wind_speed)
]

# Drop any rows with null values
# YOUR CODE HERE
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
# YOUR CODE HERE
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""

# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}
print(geoapify_key)
# Print a message to follow up on the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude, longitude = row["Lat"], row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

8880bbcb6b7f468a848236a8e8cf4216
Starting hotel search


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
# Plotting the points with the size of the point representing humidity
colormap = 'coolwarm'  # You can choose a different colormap as per your preference

# Plotting the points with the size of the point representing humidity
map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    size='Humidity',
    hover_cols=["City", "Hotel Name", "Country", "Humidity"],  # Add "Hotel Name" and "Country" to the hover tooltip
    geo=True,
    tiles='CartoLight',
    frame_width=1000,
    frame_height=650,
    color='Humidity',  # Set marker color based on humidity
    cmap='cool',  # Use the specified colormap
    colorbar=True,
    clim=(hotel_df['Humidity'].min(), hotel_df['Humidity'].max()),  # Set the colorbar range based on data
)

# Customize the plot (optional)
map_plot = map_plot.opts(title='City Hotels Map')

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)